In [364]:
import numpy as np
import pandas as pd
import random
random.uniform(0, 1)

0.3468446652009428

In [495]:
sequence=[0,1,2,3,4,3,2,1,0,4,3,2,1,2,3,4,3,2]
sequence=np.array(sequence)
intial=[0.05,0.5,0.3,0.05,0.1]
intial=np.array(intial)

In [566]:
### .............................................................Implementation of hidden markov model.................................................................
## 3 Taks to train HMM
  ## 1) Forward Algorithm
  ## 2) Backward Algorithm
  ## 3) Forward Backward Algorithm

class HMM:
    def __init__(self,sequence,initial_matrix,number_states=5,number_output=5,n_epochs=30):
        self.seq=sequence
        self.tras=np.random.uniform(0,1,(number_states,number_states))
        #self.tras=np.zeros(shape=(number_states,number_states))
        self.emiss=np.random.uniform(0,1,(number_states,number_output))
        #self.emiss=np.zeros(shape=(number_states,number_output))
        
        self.iprob=initial_matrix
        
        self.states=number_states
        
        self.out=number_output
        
        self.epochs=n_epochs
        
        self.BDP=np.zeros(shape=(number_states,len(self.seq)))
        
        self.FDP=np.zeros(shape=(number_states,len(self.seq)))
        
        self.n_epochs=n_epochs
        
    def forward(self):                                                          
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        maxi=0
        index=0                                                                     
        for j in range(nlength):
            for i in range(nstates):                         
                if j==0:
                    self.FDP[i][j]=self.iprob[i]*self.emiss[i][self.seq[j]]
                else:
                    for k in range(nstates):
                        self.FDP[i][j]+=(self.FDP[k][j-1]*self.tras[k][i]*self.emiss[i][self.seq[j]])
                    if j==nlength-1:
                        if(self.FDP[i][j]>=maxi):
                            maxi=self.FDP[i][j]
                            index=i
        tp=0.0
        for i in range(nstates):
            tp+=self.FDP[i][nlength-1]
        return tp,maxi,index
    
    
    def decoding(self):                                                           
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        tp,maxi,index=self.forward()
        hstates=np.zeros(shape=(nlength))
        for j in range(nlength-1,-1,-1):
            hstates[j]=index
            if(j==0):
                break
            for i in range(nstates):
                self.FDP[index][j]==self.FDP[i][j-1]*self.tras[i][index]*self.emiss[index][self.seq[j]]
                index=i
                break
        return hstates
    
        
    def backward(self):
        nstates=self.states                                                 
        nlength=len(self.seq)                                                     
        for j in range(nlength-1,-1,-1):
            for i in range(nstates):
                if(j==nlength-1):
                    self.BDP[i][j]=1.0
                else:
                    for k in range(nstates):
                        self.BDP[i][j]+=(self.tras[i][k]*self.emiss[k][self.seq[j+1]]*self.BDP[k][j+1])
                   
                
    def not_quite(self,time,curr_state,next_state):
        #self.forward()
        #self.backward()
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for i in range(nstates):
            deno=self.FDP[i][time]*self.BDP[i][time]
        numo=self.FDP[curr_state][time]*self.tras[curr_state][next_state]*self.emiss[next_state][self.seq[time+1]]*self.BDP[next_state][time+1]
        return numo/deno
    
    
    
    def corrected_transition(self):
        nstates=self.states                                                 
        nlength=len(self.seq)
        
        for i in  range(nstates):
            for j in range(nstates):
                numo=0
                deno=0
                for time in range(nlength-1):
                    numo+=self.not_quite(time,i,j)
                    for state in range(nstates):
                        deno+=self.not_quite(time,i,state)
                self.tras[i][j]=numo/deno
    
    
    def not_quite2(self,time,state):
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for st in range(nstates):
            deno+=self.FDP[st][time]*self.BDP[st][time]
        return self.FDP[state][time]*self.BDP[state][time]/deno
    
    
    def corrected_emmision(self):
        nstates=self.states                                                 
        nlength=len(self.seq)
        nout=self.out
        for states in (range(nstates)):
            for out in (range(nout)):
                numo=0
                deno=0
                for time in range(nlength):
                    if(self.seq[time]==out):
                        numo+=self.not_quite2(time,states)
                    deno+=self.not_quite2(time,states)
                self.emiss[states][out]=numo/deno
            
            
    def train(self):
        for epoch in range(self.n_epochs):
            print("Epoc no : {} Traning".format(epoch))
            self.forward()
            self.backward()
            self.corrected_transition()
            self.corrected_emmision()
            print("Transition Matrix")
            print(self.tras)
            print("Emmission Matrix")
            print(self.emiss)

In [567]:
# deno+=self.not_quite(time,i,states)

In [568]:
hm=HMM(sequence,intial,n_epochs=10)

In [569]:
hm.train()

Epoc no : 0 Traning
Transition Matrix
[[0.0614929  0.06954895 0.42699134 0.35080487 0.3172243 ]
 [0.03823689 0.06752621 0.51571808 0.14018135 0.06647929]
 [0.28542387 0.05199109 0.61678803 0.18190059 0.10498786]
 [0.42877476 0.04402974 0.18004633 0.09952216 0.48308175]
 [0.47989121 0.19132419 0.00494601 0.12341591 0.40692907]]
Emmission Matrix
[[0.03119538 0.12032071 0.4664339  0.13659094 0.24545907]
 [0.21128554 0.29900626 0.01165534 0.31146893 0.16658393]
 [0.24100632 0.38902642 0.31309058 0.0190038  0.03787289]
 [0.12594813 0.01792487 0.17433035 0.53507043 0.14672621]
 [0.0146466  0.05002381 0.15810857 0.54026462 0.23695639]]
Epoc no : 1 Traning
Transition Matrix
[[0.04421517 0.02919395 0.40806626 0.30674886 0.26006131]
 [0.01798677 0.0326799  0.94270681 0.01067192 0.00766649]
 [0.13658745 0.02048455 0.83247679 0.04166548 0.02744465]
 [0.35265106 0.02200936 0.20679252 0.07113454 0.39206088]
 [0.48075426 0.07931814 0.0057476  0.1175666  0.3390313 ]]
Emmission Matrix
[[0.02428972 0.10